<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-train-data" data-toc-modified-id="Loading-train-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading train data</a></span><ul class="toc-item"><li><span><a href="#Merging-train-and-test-data" data-toc-modified-id="Merging-train-and-test-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Merging train and test data</a></span></li><li><span><a href="#Merging-user-data" data-toc-modified-id="Merging-user-data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Merging user data</a></span></li><li><span><a href="#Competitions-based-features" data-toc-modified-id="Competitions-based-features-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Competitions based features</a></span></li><li><span><a href="#Competitions-data" data-toc-modified-id="Competitions-data-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Competitions data</a></span></li><li><span><a href="#Time-based-competitions-features" data-toc-modified-id="Time-based-competitions-features-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Time based competitions features</a></span></li><li><span><a href="#Current-active-competitions-feature" data-toc-modified-id="Current-active-competitions-feature-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Current active competitions feature</a></span></li><li><span><a href="#User-Interests-Feature" data-toc-modified-id="User-Interests-Feature-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>User Interests Feature</a></span></li><li><span><a href="#Submissions-based-features" data-toc-modified-id="Submissions-based-features-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Submissions based features</a></span></li><li><span><a href="#Discussion-based-features" data-toc-modified-id="Discussion-based-features-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Discussion based features</a></span></li><li><span><a href="#Comments-based-features" data-toc-modified-id="Comments-based-features-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Comments based features</a></span></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Submission" data-toc-modified-id="Submission-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Submission</a></span></li></ul></div>

In [ ]:
%load_ext autoreload
%autoreload 2

import ast
import os
import gc
import sys
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GroupKFold
from catboost import CatBoostClassifier
from category_encoders import CountEncoder
import lightgbm as lgb
import warnings
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

warnings.filterwarnings("ignore")
pd.set_option("max_colwidth", None)
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

plt.style.use('fivethirtyeight')
plt.rcParams["axes.labelsize"] = 16
plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14

In [ ]:
# class Config:
#     LAG = 3
#     VER = f'final_sub_v1'
#     OUTPUT_DIR = '.'
#     DATA_DIR = '.'
#     DEBUG = True
#     N_SPLITS = 5

In [ ]:
# LOCAL_TEST_RUN = False
seed = 0
# def seed_everything(seed=0):
random.seed(seed)
np.random.seed(seed)

# seed_everything()

# Loading train data

In [ ]:
def target(df):
    target = []
    for _, row in df.iterrows():
        if row['CompPart'] == 1:
            target.append('CompPart')
            continue
        elif row['Sub'] == 1 or row['Comment'] == 1 or row['Disc'] == 1:
            target.append('Sub')
            continue
        else:
            target.append('NoActivity')
        
    return target

In [ ]:
train = pd.read_csv("./Train.csv", index_col=None)
print(train.shape)
train['Target'] = target(train)
train.head(5)

In [ ]:
test = pd.read_csv("./Test.csv", index_col=None)
print(test.shape)
test.head(5)

In [ ]:
def define_timestamp(df):
    year_month = df['year'].astype(str) + \
        df['month'].apply(lambda x: str(x).zfill(2))
    df['year_month'] = year_month.astype(int)
    df = df.sort_values(by='year_month').reset_index(drop=True)
    df['timestamp'] = np.arange(1, len(df) + 1)
    return df

## Merging train and test data

In [ ]:
train['is_train'] = 1
test['is_train'] = 0

data = train.append(test, ignore_index=True)
timestamp = data[['year', 'month']].drop_duplicates()
timestamp = define_timestamp(timestamp)
data = data.merge(timestamp, how='left')
all_timestamps = data[['User_ID', 'timestamp', 'year', 'month']].drop_duplicates().reset_index(drop=True)

data = data.sort_values(by='timestamp').reset_index(drop=True)
data['Record'] = 1
data['Total_Num_User_Months'] = data.groupby('User_ID')['Record'].apply(lambda x: x.cumsum())

## Merging user data

In [ ]:
users = pd.read_csv("./Users.csv", index_col=None)
users.columns = ['User_ID', 'FeatureX', 'Country', 'FeatureY', 'Points', 'year', 'month', 'dayofweek']
users = users.merge(timestamp, how='left')
users = users.rename(columns={"timestamp": "Zindi_Joining_Timestamp"})
users.drop(['dayofweek', 'year', 'month', 'year_month'], axis=1, inplace=True)
users.head(5)

In [ ]:
data = data.merge(users, how='left')

columns = ['FeatureX', 'Country', 'FeatureY', 'Points']
data[columns] = data[columns].astype(str)

## Competitions based features

In [ ]:
competition_data = pd.read_csv("./CompetitionPartipation.csv", index_col=None)
competition_data.columns = [
    'CompID', 'User_ID', 'PublicRank', 'Successful_Sub_Count',
    'year', 'month', 'dayofweek'
]
    
competition_timestamp = competition_data.merge(timestamp, how='left')
competition_timestamp = competition_timestamp[[
    'User_ID', 'month', 'year', 'timestamp'
]].drop_duplicates()
competition_timestamp.columns = [
    'User_ID', 'month', 'year', 'comp_timestamp'
]
data = data.merge(competition_timestamp, how='left')

data = data.sort_values(by='timestamp').reset_index(drop=True)
data['comp_timestamp'] = data.groupby('User_ID')['comp_timestamp'].apply(lambda x: x.ffill().shift())
data['Months_Since_Last_Comp'] = data['timestamp'] - data['comp_timestamp']
data['Months_Since_Joining_Zindi'] = data['comp_timestamp'] - data['Zindi_Joining_Timestamp']

## Competitions data

In [ ]:
competitions = pd.read_csv("./Competitions.csv", index_col=None, skipinitialspace=True)
competitions['CompEndTime Year'] = [
    int(val) if val != 'not mapped' else 999 \
    for val in competitions['CompEndTime Year']
]
competitions['FeatureC'] = competitions['FeatureC'].fillna(-1).astype(np.int8)
competitions = competitions.merge(
    timestamp,
    left_on=['CompStartTime Year', 'CompStartTime Month'],
    right_on=['year', 'month'],
    how='left')
competitions = competitions.rename(columns={
    'timestamp': 'comp_start_timestamp',
})
competitions.drop(['year', 'month', 'year_month'], axis=1, inplace=True)
competitions = competitions.merge(
    timestamp,
    left_on=['CompEndTime Year', 'CompEndTime Month'],
    right_on=['year', 'month'],
    how='left')
competitions = competitions.rename(columns={
    'timestamp': 'comp_end_timestamp',
})
competitions['comp_end_timestamp'] = competitions['comp_end_timestamp'].fillna(99)
competitions.drop(['year', 'month', 'year_month'], axis=1, inplace=True)
competitions['comp_duration'] = competitions['comp_end_timestamp'] - competitions['comp_start_timestamp']

In [ ]:
for column in ['FeatureA', 'FeatureB', 'FeatureE']:
    competitions[column] = competitions[column].map(ast.literal_eval)

In [ ]:
features = competitions[['CompID']].copy()
for column in ['FeatureA', 'FeatureB', 'FeatureC', 'FeatureD', 'FeatureE']:
    table = competitions[['CompID', column]].explode(column=[column])
    table[column] = table[column].fillna('empty')
    table['count'] = 1

    table = table.pivot_table(
        index='CompID', 
        columns=column,
        values='count',
        aggfunc='count'
    )
    table.columns = [table.columns.name + "_" + str(cl) for cl in table.columns]
    table = table.reset_index()
    features = features.merge(table, how='left')
features = features.fillna(0)
features = features.merge(competitions[['CompID', 'comp_start_timestamp']])

## Time based competitions features

In [ ]:
timestamp_inds = []
competitions_inds = []

for t in tqdm(timestamp.timestamp):
    current_competition = competitions.CompID[
        (t>=competitions.comp_start_timestamp) & (t<=competitions.comp_end_timestamp)
    ]
    timestamp_inds.extend([t] * len(current_competition))
    competitions_inds.extend(current_competition)

timestamp_competition = pd.DataFrame({
    "timestamp": timestamp_inds,
    "CompID": competitions_inds
})

## Current active competitions feature

In [ ]:
current_competition = competition_data.merge(timestamp, how='left')
current_competition = current_competition.rename(columns={"timestamp": "comp_timestamp"})
current_competition = current_competition[
    ['User_ID', 'CompID', 'comp_timestamp']
].merge(competitions[
    ['CompID', 'comp_start_timestamp', 'comp_end_timestamp']
], how='left')
current_competition = current_competition[current_competition['comp_end_timestamp']!=99].reset_index(drop=True)
current_competition = all_timestamps.merge(current_competition, how='left')

current_competition['Current_Active_Competitions'] = (
    (current_competition['timestamp'] > current_competition['comp_timestamp']) &
    (current_competition['timestamp'] <= current_competition['comp_end_timestamp'])
).astype(np.int8)

current_competition = current_competition.groupby(['User_ID', 'timestamp'])['Current_Active_Competitions'].sum()
current_competition = current_competition.reset_index()

data = data.merge(current_competition, how='left')

## User Interests Feature

In [ ]:
timestamp_competition = timestamp_competition.merge(features, how='left')
timestamp_competition.drop(['comp_start_timestamp', 'CompID'], axis=1, inplace=True)
timestamp_competition = timestamp_competition.groupby('timestamp').agg(np.sum).reset_index()
timestamp_competition = timestamp_competition.sort_values('timestamp').reset_index(drop=True)

competition_timestamp = competition_data.merge(timestamp, how='left')
usr_features = competition_timestamp[['User_ID', 'CompID', 'timestamp']].merge(features, how='left')
usr_features = usr_features.drop(['CompID', 'comp_start_timestamp'], axis=1)
usr_features = usr_features.groupby(['User_ID', 'timestamp']).agg(np.sum)#.groupby(level=0).cumsum()
usr_features = usr_features.reset_index()
usr_features = all_timestamps.merge(usr_features, how='left')
columns = usr_features.columns[4:]
print(columns)

usr_features = usr_features.sort_values(by='timestamp').reset_index(drop=True)
for cl in tqdm(columns):
    usr_features[cl] = usr_features.groupby('User_ID')[cl].apply(lambda x: x.ffill())

usr_features = usr_features.fillna(0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

timestamp_inds = []
usr_inds = []
user_interests = []

for t in timestamp_competition['timestamp']:
    if t == 1:
        continue
    usr_filtered = usr_features[usr_features['timestamp']==t-1]
    timestamp_inds.extend([t]*len(usr_filtered))
    usr_inds.extend(usr_filtered.pop('User_ID'))
    
    comp_filtered = timestamp_competition[timestamp_competition['timestamp']==t]
    usr_filtered.drop(['timestamp', 'year', 'month'], axis=1, inplace=True)
    comp_filtered.drop('timestamp', axis=1, inplace=True)
    
    interests = np.matmul(usr_filtered.values, comp_filtered.values.T).flatten()
#     interests = cosine_similarity(usr_filtered.values, comp_filtered.values).flatten()
    user_interests.extend(interests)

usr_interest_f = pd.DataFrame({
    "timestamp": timestamp_inds,
    "User_ID": usr_inds,
    "user_interests": user_interests
})

data = data.merge(usr_interest_f, how='left')
data = data.sort_values(by='timestamp').reset_index(drop=True)
data['user_interests'] = data.groupby('User_ID')['user_interests'].apply(lambda x: x.ffill())
data['user_interests'] = data['user_interests'].fillna(0)

In [ ]:
history = competition_data.groupby(['User_ID', 'year', 'month'])['CompID'].nunique()
history = history.reset_index()
history.columns = [*history.columns[:-1]] + ['Num_Comp_Prev_Month']

data = data.merge(history, how='left')
data = data.sort_values(by=['User_ID', 'timestamp']).reset_index(drop=True)
data['Num_Comp_Prev_Month'] = data['Num_Comp_Prev_Month'].fillna(0)
data['Num_Comp_Per_Month'] = data.groupby('User_ID')['Num_Comp_Prev_Month'].cumsum()
data['Num_Comp_Per_Month_trend'] = data['Num_Comp_Per_Month']/data['Total_Num_User_Months']
data['Num_Comp_Per_Month_trend'] = data.groupby('User_ID')['Num_Comp_Per_Month_trend'].apply(lambda x: x.shift())
data['Num_Comp_Per_Month'] = data['Num_Comp_Per_Month']/(data['timestamp'].max() - data['Zindi_Joining_Timestamp'])
data['Num_Comp_Per_Month'] = data.groupby('User_ID')['Num_Comp_Per_Month'].apply(lambda x: x.shift())

data['Num_Comp_Prev_Month'] = data.groupby('User_ID')['Num_Comp_Prev_Month'].apply(lambda x: x.shift())
data['Num_Comp_Prev_Month_momentum'] = data['Num_Comp_Prev_Month'] - data.groupby('User_ID')['Num_Comp_Prev_Month'].apply(lambda x: x.shift(1))
data['Num_Comp_Prev_Month_momentum2'] = data['Num_Comp_Prev_Month'] - data.groupby('User_ID')['Num_Comp_Prev_Month'].apply(lambda x: x.shift(2))

In [ ]:
table = competition_data.groupby(['User_ID', 'year', 'month', 'PublicRank'])['CompID'] \
    .nunique().unstack('PublicRank').apply(lambda x: x/x.sum(), axis=1)
col_names = [table.columns.name + "_" + str(cl) for cl in table.columns]
table.columns = col_names
table = table.fillna(0)
table = table.reset_index()

table = all_timestamps.merge(table, how='left')
table = table.sort_values(by='timestamp').reset_index(drop=True)
for col in col_names:
    table[col] = table.groupby('User_ID')[col].apply(lambda x: x.cumsum().ffill().shift())
    
data = data.merge(table, how='left')

In [ ]:
table = competition_data.groupby(['User_ID', 'year', 'month', 'Successful_Sub_Count'])['CompID'].nunique().unstack('Successful_Sub_Count').apply(lambda x: x/x.sum(), axis=1)
col_names = [table.columns.name + "_" + str(col) for col in table.columns]
table.columns = col_names
table = table.fillna(0)
table = table.reset_index()

table = all_timestamps.merge(table, how='left')
table = table.sort_values(by='timestamp').reset_index(drop=True)
for col in col_names:
    table[col] = table.groupby('User_ID')[col].apply(lambda x: x.cumsum().ffill().shift())
    
data = data.merge(table, how='left')

In [ ]:
del competition_data, competition_timestamp, history
gc.collect()

## Submissions based features

In [ ]:
submission = pd.read_csv("./Submissions.csv", index_col=None)
submission.columns = ['User_ID', 'FeatureG', 'CompID', 'year', 'month', 'dayofweek']
    
submission_timestamp = submission.merge(timestamp, how='left')
submission_timestamp = submission_timestamp[['User_ID', 'month', 'year', 'timestamp']].drop_duplicates()
submission_timestamp.columns = ['User_ID', 'month', 'year', 'sub_timestamp']
data = data.merge(submission_timestamp, how='left')

data = data.sort_values(by='timestamp').reset_index(drop=True)
data['sub_timestamp'] = data.groupby('User_ID')['sub_timestamp'].apply(lambda x: x.ffill().shift())
data['Months_Since_Last_Sub'] = data['timestamp'] - data['sub_timestamp']
data['Months_Since_Sub_Joining_Zindi'] = data['sub_timestamp'] - data['Zindi_Joining_Timestamp']

In [ ]:
unique_submission = submission.groupby(['User_ID', 'year', 'month']).agg({'CompID': ['nunique', 'count']})
unique_submission.columns = ["_".join(col) for col in unique_submission.columns]
unique_submission['Sub_Per_Comp'] = unique_submission['CompID_nunique']/unique_submission['CompID_count']
unique_submission.drop(['CompID_nunique', 'CompID_count'], axis=1, inplace=True)
unique_submission = unique_submission.reset_index()
unique_submission.columns = [*unique_submission.columns[:-1]] + ['Num_Sub_Prev_Month']

data = data.merge(unique_submission, how='left')
data = data.sort_values(by=['User_ID', 'timestamp']).reset_index(drop=True)
data['Num_Sub_Prev_Month'] = data['Num_Sub_Prev_Month'].fillna(0)
data['Num_Sub_Per_Month'] = data.groupby('User_ID')['Num_Sub_Prev_Month'].cumsum()
data['Num_Sub_Per_Month_trend'] = data['Num_Sub_Per_Month']/data['Total_Num_User_Months']
data['Num_Sub_Per_Month_trend'] = data.groupby('User_ID')['Num_Sub_Per_Month_trend'].apply(lambda x: x.shift())
data['Num_Sub_Per_Month'] = data['Num_Sub_Per_Month']/(data['timestamp'].max() - data['Zindi_Joining_Timestamp'])
data['Num_Sub_Per_Month'] = data.groupby('User_ID')['Num_Sub_Per_Month'].apply(lambda x: x.shift())

data['Num_Sub_Prev_Month'] = data.groupby('User_ID')['Num_Sub_Prev_Month'].apply(lambda x: x.shift())
data['Num_Sub_Prev_Month_momentum'] = data['Num_Sub_Prev_Month'] - data.groupby('User_ID')['Num_Sub_Prev_Month'].apply(lambda x: x.shift(1))
data['Num_Sub_Prev_Month_momentum2'] = data['Num_Sub_Prev_Month'] - data.groupby('User_ID')['Num_Sub_Prev_Month'].apply(lambda x: x.shift(2))

In [32]:
submission_timestamp.head(5)

,User_ID,month,year,sub_timestamp
0,ID_8JP75F20,3,3,24
34,ID_8JP75F20,1,3,22
51,ID_8JP75F20,4,3,25
123,ID_8JP75F20,5,3,26
233,ID_8JP75F20,12,2,21


In [29]:
table = submission_timestamp.groupby(['User_ID', 'year', 'month', 'FeatureG'])['CompID'].nunique().unstack('FeatureG')#.apply(lambda x: x/x.sum(), axis=1)
col_names = [table.columns.name + "_" + str(col) for col in table.columns]
table.columns = col_names
table = table.fillna(0)
table = table.reset_index()

all_timestamps = data[['User_ID', 'timestamp', 'year', 'month']].drop_duplicates().reset_index(drop=True)
table = all_timestamps.merge(table, how='left')
table = table.sort_values(by='timestamp').reset_index(drop=True)
for col in col_names:
    table[col] = table.groupby('User_ID')[col].apply(lambda x: x.ffill().shift())
    
data = data.merge(table, how='left')
''

KeyError: 'FeatureG'

In [ ]:
del submission, submission_timestamp, unique_submission
gc.collect()

## Discussion based features

In [ ]:
usr_dis = pd.read_csv("./Discussions.csv", index_col=None)
usr_dis.columns = ['FeatureF', 'year', 'month', 'dayofweek', 'DiscID', 'User_ID']
    
usr_dis_timestamp = usr_dis.merge(timestamp, how='left')
usr_dis_timestamp = usr_dis_timestamp[['User_ID', 'month', 'year', 'timestamp']].drop_duplicates()
usr_dis_timestamp.columns = ['User_ID', 'month', 'year', 'discussion_timestamp']
data = data.merge(usr_dis_timestamp, how='left')

data = data.sort_values(by='timestamp').reset_index(drop=True)
data['discussion_timestamp'] = data.groupby('User_ID')['discussion_timestamp'].apply(lambda x: x.ffill().shift())
data['Months_Since_Last_Dis'] = data['timestamp'] - data['discussion_timestamp']
data['Months_Since_Dis_Joining_Zindi'] = data['discussion_timestamp'] - data['Zindi_Joining_Timestamp']

In [ ]:
dis_hist = usr_dis.groupby(['User_ID', 'year', 'month'])['DiscID'].nunique()
dis_hist = dis_hist.reset_index()
dis_hist.columns = [*dis_hist.columns[:-1]] + ['Num_Dis_Prev_Month']

data = data.merge(dis_hist, how='left')
data = data.sort_values(by=['User_ID', 'timestamp']).reset_index(drop=True)
data['Num_Dis_Prev_Month'] = data['Num_Dis_Prev_Month'].fillna(0)
data['Num_Dis_Per_Month'] = data.groupby('User_ID')['Num_Dis_Prev_Month'].cumsum()
data['Num_Dis_Per_Month_trend'] = data['Num_Dis_Per_Month']/data['Total_Num_User_Months']
data['Num_Dis_Per_Month_trend'] = data.groupby('User_ID')['Num_Dis_Per_Month_trend'].apply(lambda x: x.shift())
data['Num_Dis_Per_Month'] = data['Num_Dis_Per_Month']/(data['timestamp'].max() - data['Zindi_Joining_Timestamp'])
data['Num_Dis_Per_Month'] = data.groupby('User_ID')['Num_Dis_Per_Month'].apply(lambda x: x.shift())

data['Num_Dis_Prev_Month'] = data.groupby('User_ID')['Num_Dis_Prev_Month'].apply(lambda x: x.shift())
data['Num_Dis_Prev_Month_momentum'] = data['Num_Dis_Prev_Month'] - data.groupby('User_ID')['Num_Dis_Prev_Month'].apply(lambda x: x.shift(1))
data['Num_Dis_Prev_Month_momentum2'] = data['Num_Dis_Prev_Month'] - data.groupby('User_ID')['Num_Dis_Prev_Month'].apply(lambda x: x.shift(2))

In [ ]:
del usr_dis, usr_dis_timestamp, dis_hist
gc.collect()

## Comments based features

In [ ]:
usr_comments = pd.read_csv("./Comments.csv", index_col=None)
usr_comments.columns = ['User_ID', 'year', 'month', 'dayofweek']
usr_comments['CommID'] = np.arange(len(usr_comments))
    
usr_comm_timestamp = usr_comments.merge(timestamp, how='left')
usr_comm_timestamp = usr_comm_timestamp[['User_ID', 'month', 'year', 'timestamp']].drop_duplicates()
usr_comm_timestamp.columns = ['User_ID', 'month', 'year', 'comment_timestamp']
data = data.merge(usr_comm_timestamp, how='left')

data = data.sort_values(by='timestamp').reset_index(drop=True)
data['comment_timestamp'] = data.groupby('User_ID')['comment_timestamp'].apply(lambda x: x.ffill().shift())
data['Months_Since_Last_Comment'] = data['timestamp'] - data['comment_timestamp']
data['Months_Since_Comment_Joining_Zindi'] = data['comment_timestamp'] - data['Zindi_Joining_Timestamp']

In [ ]:
comm_hist = usr_comments.groupby(['User_ID', 'year', 'month'])['CommID'].nunique()
comm_hist = comm_hist.reset_index()
comm_hist.columns = [*comm_hist.columns[:-1]] + ['Num_Comm_Prev_Month']

data = data.merge(comm_hist, how='left')
data = data.sort_values(by=['User_ID', 'timestamp']).reset_index(drop=True)
data['Num_Comm_Prev_Month'] = data['Num_Comm_Prev_Month'].fillna(0)
data['Num_Comm_Per_Month'] = data.groupby('User_ID')['Num_Comm_Prev_Month'].cumsum()
data['Num_Comm_Per_Month_trend'] = data['Num_Comm_Per_Month']/data['Total_Num_User_Months']
data['Num_Comm_Per_Month_trend'] = data.groupby('User_ID')['Num_Comm_Per_Month_trend'].apply(lambda x: x.shift())
data['Num_Comm_Per_Month'] = data['Num_Comm_Per_Month']/(data['timestamp'].max() - data['Zindi_Joining_Timestamp'])
data['Num_Comm_Per_Month'] = data.groupby('User_ID')['Num_Comm_Per_Month'].apply(lambda x: x.shift())

data['Num_Comm_Prev_Month'] = data.groupby('User_ID')['Num_Comm_Prev_Month'].apply(lambda x: x.shift())
data['Num_Comm_Prev_Month_momentum'] = data['Num_Comm_Prev_Month'] - data.groupby('User_ID')['Num_Comm_Prev_Month'].apply(lambda x: x.shift(1))
data['Num_Comm_Prev_Month_momentum2'] = data['Num_Comm_Prev_Month'] - data.groupby('User_ID')['Num_Comm_Prev_Month'].apply(lambda x: x.shift(2))

In [ ]:
del usr_comments, usr_comm_timestamp, comm_hist
gc.collect()

In [ ]:
tmp = data.groupby('timestamp').agg({
    "User_ID": ["nunique"],
    "Total_Num_User_Months": ["mean", "max", "std"],
})
tmp.columns = ["_".join(col) for col in tmp.columns]
tmp = tmp.reset_index()

data = data.merge(tmp, how='left')

In [ ]:
sel_cols = ['Months_Since_Last_Comp', 'Months_Since_Last_Dis', 'Months_Since_Last_Sub', 'Months_Since_Last_Comment']
data['Months_Since_Last_Activity_Mean'] = data[sel_cols].std(axis=1)

In [ ]:
time_cols = [
    'Zindi_Joining_Timestamp',
    'comment_timestamp',
    'comp_timestamp',
    'discussion_timestamp',
    'sub_timestamp',
    'Months_Since_Last_Comp',
    'Months_Since_Last_Sub',
    'Months_Since_Last_Dis',
    'Months_Since_Last_Comment',
]

for col in time_cols:
    data[col] = data[col]/data['timestamp']

In [ ]:
tmp_time = data[data['Zindi_Joining_Timestamp']==1]
tmp_time = tmp_time.groupby('timestamp')['User_ID'].nunique().to_frame("unique_user_count")
tmp_time = tmp_time.reset_index()

data = data.merge(tmp_time, how='left')

In [ ]:
data['user_interests_rank'] = data.groupby('timestamp')['user_interests'].apply(lambda x: 
                                                                                      x.rank(method='dense', ascending=False))

In [ ]:
# data.loc[data['user_interests']==0, 'user_interests'] = np.NaN
print(train.shape, test.shape)
train, test = data[data['is_train']==1], data[data['is_train']==0]
print(train.shape, test.shape)

# Modeling

In [ ]:
def train_model(df_trainX, df_trainY, df_evalX, df_evalY, cat_cols, model_name='CAT', params=None):
    from sklearn.metrics import roc_auc_score
    if model_name == 'CAT':
        if params is None:
            params={'n_estimators':10000,'random_state':123,'cat_features':cat_cols}
        clf=CatBoostClassifier(**params,early_stopping_rounds=50,eval_metric='AUC')
        clf.fit(df_trainX,df_trainY,eval_set=(df_evalX,df_evalY),plot=False, verbose=50)
        valid_score = clf.get_best_score().get('validation').get('AUC')
        best_iteration = clf.get_best_iteration()
        feature_score = clf.get_feature_importance()
    elif model_name == 'LGB':
        if params is None:
            params={'verbose':0,'n_estimators':10000,'random_state':123,'learning_rate':0.01,'force_row_wise':True,'colsample_bytree':0.3}
        clf = lgb.LGBMClassifier(**params, importance_type='gain', metric='auc_mu', num_leaves=127, min_child_samples=5)
        callbacks = [lgb.early_stopping(500, verbose=0)]
        clf.fit(df_trainX,
                df_trainY,#)
                eval_set=[(df_evalX, df_evalY)],
                callbacks=callbacks,
                # verbose=0
               )

        valid_score = roc_auc_score(df_evalY!='NoActivity', 1-clf.predict_proba(df_evalX)[:,1])
        best_iteration = clf.booster_.best_iteration
        feature_score = clf.feature_importances_
    return clf, valid_score, best_iteration, feature_score

In [ ]:
train.to_csv(";'Train_fe.csv.gz", compression='gzip')
test.to_csv("./Test_fe.csv.gz", compression='gzip')

In [ ]:
%%time

drop_cols = [
    'year', 'month', 'Target', 'Sub', 'CompPart', 'Comment', 'Disc',
    'is_train', 'timestamp', 'Record', 'Active_Month', 'Total_Num_User_Months',
    'user_interests'
]
cat_cols = list(
    set(train.columns[train.dtypes == 'object']) - set(drop_cols) - set(['User_ID'])
)
num_cols = list(set(train.columns) - set(cat_cols + drop_cols))

train_X = train[cat_cols + num_cols]
train_X[cat_cols] = train_X[cat_cols].astype('category')
train_Y = train['Target']

test_X = test[cat_cols + num_cols]
test_X[cat_cols] = test_X[cat_cols].astype('category')

fold = GroupKFold(n_splits=5)
cb_scores, pred_cb, feat_scores = [], [], []
for it, (idxT, idxV) in enumerate(
        fold.split(train_X, train_Y, groups=train['timestamp'])):
    df_trainX, df_trainY = train_X.iloc[idxT], train_Y.iloc[idxT]
    df_evalX, df_evalY = train_X.iloc[idxV], train_Y.iloc[idxV]
    df_testX = test_X.copy()

    selected_cat_cols = ['Country']
    cat_cols_count = [f'{col}_count' for col in selected_cat_cols]
    df_trainX[cat_cols_count] = df_trainX[selected_cat_cols].copy()
    df_evalX[cat_cols_count] = df_evalX[selected_cat_cols].copy()
    df_testX[cat_cols_count] = df_testX[selected_cat_cols].copy()

    encoder = CountEncoder(cols=cat_cols_count + ['User_ID'])
    df_trainX = encoder.fit_transform(df_trainX, df_trainY)
    df_evalX = encoder.transform(df_evalX)
    df_testX = encoder.transform(df_testX)

    clf, valid_score, best_iteration, feature_score = train_model(
        df_trainX, df_trainY, df_evalX, df_evalY, cat_cols, model_name='LGB')
    cb_scores.append(valid_score)
    pred_cb.append(clf.predict_proba(df_testX)[:, 1])
    feat_scores.append(feature_score)
    print('Fold {} {} at {}'.format(it + 1, valid_score, best_iteration))

weights = cb_scores / sum(np.array(cb_scores))
print('The local CV is {}'.format(np.sum(weights * cb_scores)))

In [ ]:
# if LOCAL_TEST_RUN:
#     weights=cb_scores/sum(np.array(cb_scores))
#     print ('The local CV is {}'.format(np.sum(weights*cb_scores)))

#     prediction = np.sum(weights*np.transpose(pred_cb),1)
#     from sklearn.metrics import roc_auc_score
#     print("Test score is {}".format(roc_auc_score(test['Target']!='NoActivity', 1-prediction)))

In [ ]:
featureImp=pd.DataFrame({'feature':df_trainX.columns,'importance':np.mean(np.array(feat_scores),0)})
featureImp=featureImp.sort_values('importance',ascending=False)
featureImp['importance']=featureImp['importance']*100/featureImp['importance'].sum()
featureImp.reset_index(drop=True)

# Submission 

In [ ]:
test['Target'] = np.sum(weights * np.transpose(pred_cb), 1)
test['Target'] = 1 - test['Target']
test['UserMonthYear'] = test['User_ID'] + "_" + test['month'].astype(str) + "_" + test['year'].astype(str)
test[['UserMonthYear', 'Target']].to_csv('output.csv', index=False)

In [ ]:
!pip install catboost

In [ ]:
test[['UserMonthYear', 'Target']]